In [2]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pprint
%matplotlib inline

In [6]:
imgs = []

try:
    for i in range(1, 13):
        filename = "test" + str(i)
        #filename = "test5"
        img = cv2.imread("./images/"+filename+".jpg", cv2.IMREAD_COLOR)

        #print(img.shape)

        # image resize 360 x 219
        #img = cv2.resize(img, (500, 500), interpolation=cv2.INTER_LINEAR)

        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #cv2.imshow("img_gray", img_gray)
        #img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
        #cv2.imshow("img_blur", img_blur)

        ## 적당한 threshold 값을 찾아서
        #ret, img_th = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
        #th2 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,15,2)
        #ret1, img_th1 = cv2.threshold(img_gray, 150, 230, cv2.THRESH_BINARY_INV)
        #ret2, img_th2 = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #ret3, img_th3 = cv2.threshold(img_blur, 150, 230, cv2.THRESH_BINARY_INV)
        #ret4, img_th4 = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        img_th4 = cv2.adaptiveThreshold(img_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 5, 12)

        #img_lapl = cv2.Laplacian(img_th4, cv2.CV_32F)

        kernel = np.ones((13, 13), np.uint8)
        img_morph = cv2.morphologyEx(img_th4, cv2.MORPH_CLOSE, kernel)
        #img_lapl = cv2.Laplacian(img_morph, cv2.CV_32F)

        #cv2.imshow("image"+str(i), img_morph)
        #cv2.imshow("[morph]image"+str(i), img_morph)
        
        edges = cv2.Canny(img_morph, 50, 150, apertureSize = 3)
        minLineLength = 50
        maxLineGap = 10
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength, maxLineGap)
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(img, (x1,y1), (x2,y2), (0,255,0), 1)
        
        cv2.imshow("edges", edges)
        cv2.imshow("result", img)
        
        #print(lines)
        #print()

        # 대상은 흰색, 배경은 까만색이어야 컨투어 찾기 가능
        contours, hierachy = cv2.findContours(img_morph.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #img_cont = cv2.drawContours(img.copy(), contours, -1, (0,255,0), 1)
        
        #cv2.imshow("[cont]image"+str(i), img_cont)
        
        ###################### boundingRect test [start] ######################
        #x, y, w, h = cv2.boundingRect(contours[0])
        #print(x, y, w, h)
        #testimg = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        #cv2.imshow("img"+str(i), testimg)
        ###################### boundingRect test [end] ########################
        
        # 사각형 그리기
        rects = [cv2.boundingRect(each) for each in contours]

        tmp = [w*h for (x,y,w,h) in rects]
        tmp.sort()

        # 제일 큰 아이 두개를 빼고, 나머지의 넓이를 찾을 수 있습니다. 그래서 넓이값으로 다시 한 번 더 걸러주는거죠...
        #rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>15000)and(w*h<500000))]
        rects = [(x,y,w,h) for (x,y,w,h) in rects]

        img_result = []
        img_for_class = img.copy()

        margin_pixel = 30

        for rect in rects:
            #[y:y+h, x:x+w]
            img_result.append(
                img_for_class[rect[1]-margin_pixel : rect[1]+rect[3]+margin_pixel, 
                              rect[0]-margin_pixel : rect[0]+rect[2]+margin_pixel])

            # Draw the rectangles
            cv2.rectangle(img, (rect[0], rect[1]), 
                          (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 2)

        imgs.append(img)
        
except:
    print("Exception!!!!!!!!!")
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cv2.waitKey(0)
cv2.destroyAllWindows()

Exception!!!!!!!!!


In [22]:
# 이미지 한번에 출력
for i in range(0, len(imgs)):
    cv2.imshow("image"+str(i), imgs[i])
    
cv2.waitKey(0)
cv2.destroyAllWindows()

In [214]:
tmp = [w*h for (x,y,w,h) in rects]
tmp.sort()
tmp

[374, 1254, 1474, 2646, 3165, 3420, 3500, 4150, 4700, 7303, 14348, 14348]

In [215]:
# 제일 큰 아이 두개를 빼고, 나머지의 넓이를 찾을 수 있습니다. 그래서 넓이값으로 다시 한 번 더 걸러주는거죠...
#rects = [(x,y,w,h) for (x,y,w,h) in rects if ((w*h>15000)and(w*h<500000))]
rects = [(x,y,w,h) for (x,y,w,h) in rects]
rects

[(345, 292, 66, 19),
 (96, 261, 45, 76),
 (270, 239, 67, 109),
 (85, 223, 67, 22),
 (163, 174, 35, 100),
 (354, 167, 47, 100),
 (22, 162, 22, 17),
 (91, 147, 49, 54),
 (279, 146, 50, 83),
 (420, 138, 68, 211),
 (201, 138, 15, 211),
 (11, 138, 68, 211)]

In [216]:
img_result = []
img_for_class = img.copy()

margin_pixel = 60
print(rects)
for rect in rects:
    #[y:y+h, x:x+w]
    img_result.append(
        img_for_class[rect[1]-margin_pixel : rect[1]+rect[3]+margin_pixel, 
                      rect[0]-margin_pixel : rect[0]+rect[2]+margin_pixel])
    
    # Draw the rectangles
    cv2.rectangle(img, (rect[0], rect[1]), 
                  (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 5)

#plt.figure(figsize=(15,12))
#plt.imshow(img)
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[(345, 292, 66, 19), (96, 261, 45, 76), (270, 239, 67, 109), (85, 223, 67, 22), (163, 174, 35, 100), (354, 167, 47, 100), (22, 162, 22, 17), (91, 147, 49, 54), (279, 146, 50, 83), (420, 138, 68, 211), (201, 138, 15, 211), (11, 138, 68, 211)]
